# Frequent Itemset generation for Big and Wide Data

The idea is to make initial recommendations for each user using Matrix Factorisation, and then find intersection of users using simple set operations to generate frequent itemsets. The initial recommendations are made using implicit's ALS algorithm, which recommends one most-suitable item per user. 

In [1]:
!pip install mlxtend
!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import implicit
import random

In [3]:
from mlxtend.preprocessing import OnehotTransactions
from scipy.sparse import csr_matrix
from math import floor, ceil

# Data

The example data set is available at : http://archive.ics.uci.edu/ml/datasets/online+retail. It has been extracted and stored in excel format.

The data is One-hot encoded using mlxtends OnehotTransactions(). This gives a sparse dataframe with columns as products and customers as rows. The data set contains 1204 transactions of 2604 products, and hence it is Wide and Big data.

In [4]:
data = pd.read_excel("Online_Retail.xls")

In [5]:
g = data.groupby(["CustomerID"], as_index = False)
data.fillna( value = '-', inplace = True )

In [6]:
Itemset = []
user = []
for item in list(g.groups.keys()) :
    Itemset.append(list(g.get_group(item)["Description"]))
    user.append(item)

In [7]:
oht = OnehotTransactions()
u = oht.fit(Itemset).transform(Itemset)

In [8]:
Matrix = pd.DataFrame(u, columns = oht.columns_)
Matrix["user"] = user
Matrix = Matrix.set_index("user")

In [9]:
Matrix.head()

,4 PURPLE FLOCK DINNER CANDLES,OVAL WALL MIRROR DIAMANTE,SET 2 TEA TOWELS I LOVE LONDON,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC FINISH 15CM PLANTER POTS,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
user,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12356.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12359.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
len(Matrix)

1204

The following table is a user-item matrix with rating scale intrinsically based on quantity of items purchased.

In [11]:
table = data.pivot_table(index = 'CustomerID', columns = 'Description', values = 'Quantity', aggfunc = np.mean)

In [12]:
table = table.fillna(0)

In [13]:
table.head()

Description,4 PURPLE FLOCK DINNER CANDLES,OVAL WALL MIRROR DIAMANTE,SET 2 TEA TOWELS I LOVE LONDON,*Boombox Ipod Classic,*USB Office Mirror Ball,-,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,...,ZINC WILLIE WINKIE CANDLE STICK,amazon,amazon sales,check,counted,damages,faulty,found,"mouldy, thrown away.",reverse 21/5/10 adjustment
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12356.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12359.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
index_names = list(table.index)

# Initial Recommendations

In [31]:
K = csr_matrix(table)

In [32]:
model = implicit.als.AlternatingLeastSquares(factors=500)
model.fit(K.T)

In [33]:
user_items = K.tocsr()

In [34]:
rr = []
for item in range(0, len(table), 1) :
    u = model.recommend(item, user_items)
    rr.append(u[0])

In [35]:
Recc = pd.DataFrame(rr, columns = ["item", "del"])
Recc = Recc.drop("del", axis = 1)
Recc["User"] = index_names
Recc = Recc.set_index("User")

In [36]:
Recc.head()

,item
User,
12346.0,972
12347.0,209
12348.0,209
12356.0,903
12359.0,552


# Generating frequent itemsets

a) This function finds users who have bought a set of products. It returns a list of CustomerIDs associated with the purchase of 'products'. As the lendth of the list 'products' grows, size of intersection of users decerases drastically (due to large dimentionality of data). This function is responsible for clustering users.  

In [37]:
def find_intersection_of_users( products ) :
    
    u = []
    group = []
    l = len(products)
    
    # For one/a set of popular items in a, find indices of users : iSEC
    
    if l > 1 :
        for item in products :
            u.append( Matrix[Matrix[item] == 1].index.tolist() )
        
        iSEC = set()
        for jaytem in range(0, l, 1) :
            if jaytem == 0 :
                iSEC = set(u[jaytem]).intersection(u[jaytem + 1])
            if jaytem >= 2 : 
                iSEC = iSEC.intersection(u[jaytem])
    
        group = list(iSEC)
    
    else : group = Matrix[Matrix[products[0]] == 1].index.tolist()
    
    return group

b) This function takes in the intersection of users products recommended by the ALS algorithm. For example, if there are 20 users who've bought 'products', then this function returns 20 potential frequent itemsets, each of which has increased in length by 1. This function is responsible for growth of the Frequent Itemset.

In [38]:
def iter_itemset( the_group, products ) :
    
    result = []
    
    if len(the_group) > 4 :
        
        for item in the_group :
            u = [table.columns[int(Recc.xs(item))]] + products
            #print(u)
            result.append(u)
        
    else : print("Sample populaiton has fallen below 5!")
            
    return result

c) This function runs the iterations

In [39]:
def loop( seed ) :
    gpx = find_intersection_of_users(seed)
    potential_itemset = iter_itemset(gpx, seed)
    
    return potential_itemset

# Example 

Let 'x' be the seed product that we start the iterations with.

In [40]:
x = ['DOLLY GIRL LUNCH BOX']

In [41]:
K = loop(x)
K

[['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ENCHANTED BIRD COATHANGER 5 HOOK', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ENCHANTED BIRD COATHANGER 5 HOOK', 'DOLLY GIRL LUNCH BOX'],
 ['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['CHALKBOARD KITCHEN ORGANISER', 'DOLLY GIRL LUNCH BOX'],
 ['CHALKBOARD KITCHEN ORGANISER', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['SPACEBOY TV DIN

In [42]:
l = floor(len(K)/2)
l

30

In [43]:
J = random.sample(K, l)
J

[['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['CHALKBOARD KITCHEN ORGANISER', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['SPACEBOY TV DINNER TRAY', 'DOLLY GIRL LUNCH BOX'],
 ['WHITE CHERRY LIGHTS', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ENCHANTED BIRD COATHANGER 5 HOOK', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN', 'DOLLY GIRL LUNCH BOX'],
 ['ASSTD DESIGN RACING CAR PEN',

In [45]:
for item in range(0, l, 1) :
    J[item] = loop(J[item])

Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!
Sample populaiton has fallen below 5!


In [46]:
J

[[['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ["PAPER CHAIN KIT 50'S CHRISTMAS ",
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ASSTD DESIGN RACING CAR PEN',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX']],
 [],
 [],
 [],
 [['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ["PAPER CHAIN KIT 50'S CHRISTMAS ",
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ASSTD DESIGN RACING CAR PEN',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX']],
 [['ENCHANTED BIRD COATHA

In [47]:
J = [x for x in J if x != []]

In [48]:
J

[[['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ["PAPER CHAIN KIT 50'S CHRISTMAS ",
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ASSTD DESIGN RACING CAR PEN',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX']],
 [['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ["PAPER CHAIN KIT 50'S CHRISTMAS ",
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ASSTD DESIGN RACING CAR PEN',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX']],
 [['ENCHANTED BIRD COATHANGER 5 HOOK',
 

In [49]:
l = floor(len(J)/2)
J = random.sample(J, l)

In [50]:
J

[[['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ["PAPER CHAIN KIT 50'S CHRISTMAS ",
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ASSTD DESIGN RACING CAR PEN',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX'],
  ['ENCHANTED BIRD COATHANGER 5 HOOK',
   'SPACEBOY TV DINNER TRAY',
   'DOLLY GIRL LUNCH BOX']],
 [['ASSTD DESIGN RACING CAR PEN',
   "PAPER CHAIN KIT 50'S CHRISTMAS ",
   'DOLLY GIRL LUNCH BOX'],
  ['SPACEBOY TV DINNER TRAY',
   "PAPER CHAIN KIT 50'S CHRISTMAS ",
   'DOLLY GIRL LUNCH BOX'],
  ['ASSTD DESIGN RACING CAR PEN',
   "PAPER CHAIN KIT 50'S CHRISTMAS ",
   'DOLLY GIRL LUNCH BOX'],
  ['CHALKBOARD KITCHEN ORGANISER',
   "PAPER CHAIN KIT 50'S CHRISTMAS ",
   'DOLLY GIRL LUNCH BOX'],
  ['SPACEBOY TV DINNER TRAY',
   "PAPER CHAIN KIT 50'S CHRISTMAS ",
   'DOLLY GIRL LUNCH BOX'],
  ['DOORMAT WELCOME PUPPIES

# ... and so on.

The result is exponential reduction in comparisons due to sparse data and random sampling of the potential itemsets. 